In [2]:
import numpy as np
import pandas as pd
import datetime
import math

In [92]:
flow = pd.read_csv('flow.csv')
flow['date']=pd.to_datetime(flow['flowdate'],format="%Y-%m-%d")#"%d/%m/%Y")
flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index(pd.DatetimeIndex(flow['date']))
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek
flow['month_no']=flow['year']+flow['month']/100
flow['hour']=flow['hour']
flow['hour_ref'] = flow.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)

flow

,hour,flow,from,to,date,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,,,
2016-01-02,13,50,BOT,ZIMA,2016-01-02,2016,1,2,5,2016.01,weekend_13
2016-01-02,14,63,ZIMA,ZIM,2016-01-02,2016,1,2,5,2016.01,weekend_14
2016-01-02,15,263,ZIM,ZAMZ,2016-01-02,2016,1,2,5,2016.01,weekend_15
2016-01-02,16,100,MOZN_EDM,ZIMA,2016-01-02,2016,1,2,5,2016.01,weekend_16
2016-01-02,17,120,ZIM,ZAMZ,2016-01-02,2016,1,2,5,2016.01,weekend_17
...,...,...,...,...,...,...,...,...,...,...,...
2018-09-10,8,45,ZIM,ZAMZ,2018-09-10,2018,9,10,0,2018.09,weekday_8
2018-09-10,9,190,MOZS,RSAN,2018-09-10,2018,9,10,0,2018.09,weekday_9
2018-09-10,10,68,RSAS,NAM,2018-09-10,2018,9,10,0,2018.09,weekday_10


In [93]:
flowa=flow.copy().rename(columns={'to':'from','from':'to'})
flow.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    ) 
    ,['to','from']]=flowa.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    )     
    ,['to','from']]

In [94]:
f_to=flow[['hour','flow','to','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'to': 'node'})
f_from=flow[['hour','flow','from','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']

flows=pd.concat([f_to, f_from])

flows

,hour,flow,node,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,
2016-01-02,13,50,ZIMA,2016,1,2,5,2016.01,weekend_13
2016-01-02,14,63,ZIM,2016,1,2,5,2016.01,weekend_14
2016-01-02,15,263,ZAMZ,2016,1,2,5,2016.01,weekend_15
2016-01-02,16,100,MOZN_EDM,2016,1,2,5,2016.01,weekend_16
2016-01-02,17,120,ZAMZ,2016,1,2,5,2016.01,weekend_17
...,...,...,...,...,...,...,...,...,...
2018-09-10,8,-45,ZIM,2018,9,10,0,2018.09,weekday_8
2018-09-10,9,-190,RSAN,2018,9,10,0,2018.09,weekday_9
2018-09-10,10,-68,RSAS,2018,9,10,0,2018.09,weekday_10


In [95]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='MOZ'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [96]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week','month_no','hour_ref']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows

,date,hour,node,year,month,day,day_of_week,month_no,hour_ref,flow
0,2016-01-01,0,ZAMC,2016,1,1,4,2016.01,weekday_0,160
1,2016-01-01,0,ZAMZ,2016,1,1,4,2016.01,weekday_0,10
2,2016-01-01,0,ZIM,2016,1,1,4,2016.01,weekday_0,-170
3,2016-01-01,1,ZAMC,2016,1,1,4,2016.01,weekday_1,160
4,2016-01-01,1,ZAMZ,2016,1,1,4,2016.01,weekday_1,10
...,...,...,...,...,...,...,...,...,...,...
137806,2018-09-10,10,BOT,2018,9,10,0,2018.09,weekday_10,185
137807,2018-09-10,10,MOZ,2018,9,10,0,2018.09,weekday_10,173
137808,2018-09-10,10,NAM,2018,9,10,0,2018.09,weekday_10,68
137809,2018-09-10,10,RSA,2018,9,10,0,2018.09,weekday_10,-467


In [97]:
# flows.loc[(flows['date']=='2016-01-15') & (flows['hour']==0)]

In [98]:
flow_monthly=flows.groupby(['month_no','year','month','hour', 'hour_ref','node']).sum().reset_index(['hour','month','year'])
flow_monthly.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly=flow_monthly.drop(['day_of_week','day'],axis=1)
flow_monthly

year  month  hour  flow_sum
month_no hour_ref   node                             
2016.01  weekday_0  BOT   2016      1     0       -71
                    MOZ   2016      1     0      1319
                    NAM   2016      1     0        50
                    RSA   2016      1     0      -104
                    ZAMC  2016      1     0      1391
...                        ...    ...   ...       ...
2018.09  weekend_23 NAM   2018      9    23       763
                    RSA   2018      9    23     -1367
                    ZAMC  2018      9    23       162
                    ZAMZ  2018      9    23      -195
                    ZIM   2018      9    23       -60

[10009 rows x 4 columns]

In [90]:
fm=flow_monthly.reset_index()

fm.groupby(['month_no','hour_ref']).sum().sort_values('flow_sum')

fm.loc[(fm['month_no']==2017.01) & (fm['hour_ref']=='weekend_20')]['flow_sum'].sum()

0

In [91]:
flow_monthly.to_csv('dam_flow_monthly.csv')